# Day 7

In [1]:
data = open(
    '/content/drive/MyDrive/Colab Notebooks/AOC/2022/day7.txt',
    'r').readlines()

In [2]:
print(data[:10])

['$ cd /\n', '$ ls\n', '282959 btm\n', 'dir fmfnpm\n', 'dir gwlwp\n', 'dir hchp\n', '275929 hmbbjbf\n', 'dir nsphznf\n', 'dir phschqg\n', '193293 rhpwvff\n']


## Part 1

In [17]:
from __future__ import annotations
from typing import Optional

class Directory:

    def __init__(self, name: str, parent: Optional[Directory] = None):
        self.name = name
        self.parent = parent
        self.subdirs = {}   # subdirs must be unique in any given dir
        self.files = []
        self.size = 0

    def add_subdir(self, subdir: str):
        self.subdirs[subdir] = Directory(subdir, self.name)

    def add_file(self, file: str, size: str):
        self.files.append((file, size))
        self.size += int(size)

    def compute_size(self):
        for dir in self.subdirs.values():
            self.size += dir.compute_size()

        return self.size

    def __repr__(self):
        return f"({self.name}, {self.parent})"

In [22]:
from typing import List

def findDirectoriesBySize(dir: Directory,
                          size: int,
                          ans: List[int] = None) -> List[int]:
    """
    Helper method to recursively find all subdirectories up
    to specified size and returns a list of their sizes
    """
    if ans is None:
        ans = []
    for d in dir.subdirs.values():
        if d.size <= size:
            ans.append(d.size)
        if len(d.subdirs) > 0:
            findDirectoriesBySize(d, size, ans)

    return ans

def findTotalSizeOfDirectoriesP1(input: List[str]) -> int:
    root = Directory("/")
    path = [root]
    for line in input[2:]:
        l = line.rstrip().split(" ")
        prev = path[-1]
        if l[0] == "dir":
            prev.add_subdir(l[1])
            # print(f"Added a subdirectory, {l[1]} into current directory {prev}")
        elif l[1] == "cd" and l[2] != '..':
            path.append(prev.subdirs.get(l[2]))
            # print(f"Cd into {l[2]} from current directory {prev}")
        elif l[0].isnumeric():
            prev.add_file(l[1], l[0])
            # print(f"Added a file, {l[1]} to current directory {prev}")
        elif l[1] == "cd" and l[2] == '..':
            path.pop()

    root.compute_size()
    print(f"Size of root directory: {root.size}")
    sizes = findDirectoriesBySize(root, 100000)
    return sum(sizes)


In [23]:
findTotalSizeOfDirectoriesP1(data)

Size of root directory: 46090134


1491614

## Part 2

In [24]:
AVAILABLE = 70000000    # Disk size
MIN_FREE = 30000000     # Minimum free space

def findDirectoriesBySize(dir: Directory,
                          size: int,
                          ans: List[int] = None) -> List[int]:
    """
    Helper method to recursively find all subdirectories at least
    as large as specified size and returns a list of their sizes
    """
    if ans is None:
        ans = []
    for d in dir.subdirs.values():
        if d.size >= size:
            ans.append(d.size)
        if len(d.subdirs) > 0:
            findDirectoriesBySize(d, size, ans)

    return ans

def findSmallestDirToDelete(input: List[str]) -> int:
    root = Directory("/")
    path = [root]
    for line in input[2:]:
        l = line.rstrip().split(" ")
        prev = path[-1]
        if l[0] == "dir":
            prev.add_subdir(l[1])
        elif l[1] == "cd" and l[2] != '..':
            path.append(prev.subdirs.get(l[2]))
        elif l[0].isnumeric():
            prev.add_file(l[1], l[0])
        elif l[1] == "cd" and l[2] == '..':
            path.pop()

    root.compute_size()
    free = AVAILABLE - root.size
    min_size = MIN_FREE - free
    sizes = findDirectoriesBySize(root, min_size)
    sizes.sort()

    return sizes[0]

In [25]:
findSmallestDirToDelete(data)

6400111